### Load Data

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

#other
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

import scipy

import torch
from sentence_transformers import SentenceTransformer, models

In [ ]:
data_np_nostop=np.load('../../data/data_np_nostop.npy', allow_pickle=True)

In [6]:
data=np.load('../../data/cleaned2.npy', allow_pickle=True)

### Load the model

#### CamemBERT

In [ ]:
word_embedding_model = models.Transformer('flaubert/flaubert_large_cased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
import time
t = time.time()
embeddings= model.encode(data[:1000])
elapsed= time.time()-t
print(elapsed)

#### Multilingual

In [7]:
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [8]:
embeddings= model.encode(data, batch_size=8, show_progress_bar=True)

In [9]:
np.save('../../data/multilingual_embeddings.npy', embeddings, allow_pickle=True)

In [ ]:
data[245]

#### PCA

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=30)
pca.fit(embeddings)
embeddings_pca=pca.transform(embeddings)

In [ ]:
embeddings_pca.shape

In [ ]:
data[7229]

### Queries based on cosine

In [13]:
import pandas as pd
df=pd.read_csv('../../data/querys.csv', sep=',',error_bad_lines=False, encoding='latin-1') #le fichier original

In [23]:
queries=np.array(df['query'], dtype=str)

In [24]:
queries

array(['siren', 'sirene', 'entreprise', 'entreprises', 'siret',
       'open damir', 'opendamir', 'damir', 'contours départements',
       'emissions polluantes', 'géofla départements',
       'effectifs police municipale', 'marchés public bourgogne',
       'Liste gares SNCF', 'contours départements français',
       'loi de finance 2016', 'lolf 2016', 'formations pas de calais',
       'accidents de la circulation', 'accidents de la route',
       'risque de décès un an après accident', 'COG',
       'code officiel géographique', 'contour commune',
       'contours communes', 'contour communes', 'code postal',
       'codes postaux', 'prénoms', 'association', 'associations', 'RNA',
       'nan', 'répertoire des associations',
       'répertoire national des associations', 'waldec',
       'organismes de formation', 'organisme de formation',
       'bibliothèques', "annuaire de l'éducation", 'grand débat',
       'vie-publique répertoire'], dtype='<U36')

In [25]:
#queries = ['répertoire association', 'base adresses nationale']
query_embeddings = model.encode(queries)
#query_embeddings_pca=pca.transform(query_embeddings)
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 10
for query, query_embedding in zip(queries, query_embeddings):#_pca):
    distances = scipy.spatial.distance.cdist([query_embedding], embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(idx)
        print(data [idx], "(Score: %.4f)" % (1-distance))






Query: siren

Top 5 most similar sentences in corpus:
7897
sirene. Mairie d'ANDUZE. sirene. nan (Score: 0.6412)
17537
zef. nan. sef. nan (Score: 0.5367)
6534
erge. nan. erge. nan (Score: 0.5302)
12982
bli. nan. foo. nan (Score: 0.5109)
20963
rez. nan. rez. nan (Score: 0.5014)
21320
o. nan. o. nan (Score: 0.4989)
21007
 . nan. . nan (Score: 0.4883)
36488
. nan. esdess. nan (Score: 0.4854)
7629
aaa. nan. aaa. nan (Score: 0.4852)
37086
aaa. nan. aaa. nan (Score: 0.4852)




Query: sirene

Top 5 most similar sentences in corpus:
7897
sirene. Mairie d'ANDUZE. sirene. nan (Score: 0.7128)
36488
. nan. esdess. nan (Score: 0.5539)
6534
erge. nan. erge. nan (Score: 0.5404)
17537
zef. nan. sef. nan (Score: 0.5400)
7629
aaa. nan. aaa. nan (Score: 0.4917)
37086
aaa. nan. aaa. nan (Score: 0.4917)
20963
rez. nan. rez. nan (Score: 0.4852)
21320
o. nan. o. nan (Score: 0.4850)
21007
 . nan. . nan (Score: 0.4771)
37587
ot. nan. ot. nan (Score: 0.4759)




Query: entreprise

Top 5 most similar sentenc





Query: emissions polluantes

Top 5 most similar sentences in corpus:
11940
emissions de polluants des départements tous secteurs d'activité confondus dans la région sud. AtmoSud. estimation des émissions annuelles totales des principaux polluants atmosphériques so2 - nox équivalent no2 - particules en suspension - particules en suspension pm2.5 - benzène - composés organiques volatils non méthaniques covnm - ammoniac nh3 - monoxyde de carbone co - as - cd - ni - pb – bap sur la région sud pour les années et . toutes les données fournies sont kg. le nombre de décimales varie en fonction du polluant. les valeurs d'émissions de polluants atmosphériques ont été calculées conformément au guide méthodologique pour l'élaboration des inventaires territoriaux des émissions atmosphériques réalisé par le pôle de coordination national sur les inventaires d'émissions territoriaux pcit et au référentiel français ominea élaboré par le citepa. la classification des secteurs d'activité est la class





Query: marchés public bourgogne

Top 5 most similar sentences in corpus:
40893
marchés publics france . . marchés publics en france en . france marches-public (Score: 0.5913)
42588
marchés publics france . . marchés publics en france en . france marches-public (Score: 0.5913)
1842
marchés publics - angers - . Région des Pays de la Loire. etat annuel des marchés notifiés de plus de 20 € entre le 1er janvier et le 31 décembre par la ville d'angers. information diffusée en respect de l'article du code des marchés publics. id marche : identifiant du marché objet du marche : objet du marché objet du marche bis : descriptions complétaires de l'objet du marché titulaire marche nom: raison sociale du titulaire du marché titulaire marche cp : code postal du titulaire du marché titulaire marche ville : ville du titulaire du marché nature marche : travaux services fournitures tranche budgetaire : tranche budgétaire. citoyennete-institution commande-public marche-public (Score: 0.4959)
15422
m





Query: loi de finance 2016

Top 5 most similar sentences in corpus:
40499
projet de loi de finances pour plf recettes fiscales nettes. Ministère de l'économie et des finances. présentation des évaluations des recettes fiscales du projet de lois de finances projet de lois de finances plf en million d'euros.. budget-de-letat finances-publiques loi-de-finances  (Score: 0.4600)
42395
projet de loi de finances pour plf recettes fiscales nettes. Ministère de l'économie et des finances. recettes nettes du budget de l'etat présentées au plf . budget-de-letat finances-publiques loi-de-finances  (Score: 0.4565)
41854
projet de loi de finances pour plf recettes fiscales nettes. Ministère de l'économie et des finances. les recettes nettes du budget de l'etat présentées au projet de lois de finances plf . annexes-budgetaires budget-de-letat finances-publiques loi-de-finances  (Score: 0.4554)
5286
loi de finances initiale pour lfi . Ministère de l'économie et des finances. crédits en autorisatio





Query: COG

Top 5 most similar sentences in corpus:
208
cc. nan. cc. nan (Score: 0.6613)
8712
gjgj. nan. gfjfj. nan (Score: 0.6154)
37547
dfg. nan. dfg. nan (Score: 0.5729)
21125
dgdfg. nan. dfgd. nan (Score: 0.5649)
37035
sdgf. nan. sdfg. nan (Score: 0.5577)
33415
jkljkl. nan. fghgjghj. nan (Score: 0.5520)
10640
decp. nan. decp. decp (Score: 0.5498)
374
okokokk. nan. okokokko. nan (Score: 0.5487)
26725
tetst. nan. kok. nan (Score: 0.5423)
37587
ot. nan. ot. nan (Score: 0.5407)




Query: code officiel géographique

Top 5 most similar sentences in corpus:
41014
codes insee communes géolocalisées. . nan. code commune geolocalise insee (Score: 0.6500)
42627
codes insee communes géolocalisées. . nan. code commune geolocalise insee (Score: 0.6500)
19917
code officiel géographique : liste des communes existantes au janvier . Région Île-de-France. le code officiel géographique rassemble les codes et libellés des communes des cantons des arrondissements des départements des régions des co





Query: codes postaux

Top 5 most similar sentences in corpus:
40935
api codes postaux. . nan. codes-postaux (Score: 0.7961)
42634
api codes postaux. . nan. codes-postaux (Score: 0.7961)
25061
les codes postaux. Ville de Grenoble. les codes postaux du territoire grenoblois. code-postal (Score: 0.6477)
11592
code postal = code insee. nan. liste des codes postaux dont la valeur est aussi une valeur de code insee. d'après les données : datasets base-officielle-des-codes-postaux . code-insee code-postal (Score: 0.5765)
37221
base officielle des codes postaux. La Poste. fichier de correspondance entre les codes communes insee et les codes postaux au format csv. ce fichier comprend : - le code commune insee - le nom de la commune - le code postal - le libellé d’acheminement - la ligne de l'adresse précision de l'ancienne commune ou du lieu-dit il correspond aux codes postaux de france métropole et dom ceux des tom ainsi que monaco. note aux réutilisateurs : les contours géographiques des 





Query: répertoire des associations

Top 5 most similar sentences in corpus:
33366
statuts association. nan. exemples de statuts d'associations. nan (Score: 0.5508)
37884
membres associatifs. nan. localiser les membres de l'association. nan (Score: 0.5243)
33963
associations. Ville de Vanves. liste des associations vanvéennes. associations vanves (Score: 0.5094)
11514
guide des associations de petit-couronne . Mairie de Petit-Couronne. guide des associations de petit-couronne . nan (Score: 0.4433)
2449
liste des associations décembre . Mairie de Saint-Marcel 71 . liste des associations décembre . nan (Score: 0.4427)
36927
associations sur ville d'avray. Commune de Ville d'Avray. liste des associations oeuvrant sur la commune de ville d'avray. nan (Score: 0.4402)
19344
associations - les données du journal officiel des associations et fondations d'entreprise. Région Île-de-France. les données ''associations'' sont extraites du joafe journal officiel des associations et fondations d'e





Query: organismes de formation

Top 5 most similar sentences in corpus:
7294
education . CORBION. education.. nan (Score: 0.4802)
26724
formation wordpress. nan. formation wordpress. nan (Score: 0.4596)
9113
collèges. ville d'Agen. nan. colleges ecole education enseignement formation recherche (Score: 0.4291)
6880
formations organisées par la médiathèque départementale en . Région Occitanie Pyrénées Méditerranée. liste des formations organisées par la médiathèque départementale de la haute-garonne destinations des bibliothécaires municipaux de son réseau par année commune et catégories d’agents concernés. l’accès aux formations est gratuit.. apprentissage bibliotheque culture (Score: 0.4257)
9083
lycées. ville d'Agen. nan. education enseignement formation recherche (Score: 0.4245)
26099
formations organisées par la médiathèque départementale en . Conseil départemental de Haute-Garonne. liste des formations organisées par la médiathèque départementale de la haute-garonne destination

### Exploration

In [ ]:
len(embeddings[0])

In [ ]:
print(data_np_nostop[:1000])

### KD Tree

In [ ]:
from scipy import spatial
A = embeddings
tree = spatial.KDTree(A)

In [ ]:
queries = ['historique météorologique', 'station météo', "budget régional d'assurance maladie"]
query_embeddings = model.encode(queries)
#query_embeddings_pca=pca.transform(query_embeddings)
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 10
for query, query_embedding in zip(queries, query_embeddings):
    distances, results=tree.query(query_embedding, k=closest_n)

    print("\n\n======================\n\n")
    print("Query:", query)
    print(f"\nTop {closest_n} most similar sentences in corpus:")

    for idx, distance in zip(results, distances):
        print(data_np_nostop[idx], "(Score: %.4f)" % (1-distance/50))

### Official example

In [ ]:
word_embedding_model = models.Transformer('flaubert/flaubert_large_cased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedder = SentenceTransformer('distiluse-base-multilingual-cased')

# Corpus with example sentences
corpus = ['Un homme mange de la nourriture',
          "Un homme mange un morceau de pain.",
          "Un homme mange des pâtes.",
          "La fille porte un bébé.",
          "Le bébé est porté par la femme",
          'Un homme est à cheval',
          "Un homme monte un cheval blanc sur un terrain clos.",
          "Un singe joue du tambour.",
          "Un gorille joue du tambour.", "Un singe joue du tambour.",
          "Un guépard court derrière sa proie.",
          "Un guépard poursuit sa proie à travers un champ."
          ]
corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

### Extract Vectors

In [ ]:
vectors=[]
for i in range(1000):
    doc=nlp(data_np_nostop[i])
    vectors_doc=[]
    for token in doc:
        vectors_doc.append(token.vector)
    vectors.append(np.array(vectors_doc))

In [ ]:
vectors=np.array(vectors)

In [ ]:
vectors[2].shape

In [ ]:
len(vectors_clean)

In [ ]:
import random 
rand= random.randint(1, len(data_np_nostop)//10)
print(rand)
print(np.var(vectors[rand]))

from sklearn import decomposition
pca = decomposition.PCA(n_components=2)
pca.fit(vectors[rand])
vectors_pca=pca.transform(vectors[rand]).transpose()
x,y = np.mean(vectors_pca[0]), np.mean(vectors_pca[1])

plt.figure(0)
plt.scatter(vectors_pca[0], vectors_pca[1])


from sklearn.ensemble import IsolationForest
clf = IsolationForest(random_state=0, contamination=0.4).fit(vectors[rand])
truth=clf.predict(vectors[rand])

vectors_clean=[]
for i in range(len(vectors[rand])):
    if truth[i]==1:
               vectors_clean.append(vectors[rand][i])


from sklearn import decomposition
pca = decomposition.PCA(n_components=2)
pca.fit(vectors_clean)
vectors_pca_clean=pca.transform(vectors_clean).transpose()
xc,yc = np.mean(vectors_pca_clean[0]), np.mean(vectors_pca_clean[1])
#plt.figure(1)
#plt.scatter(vectors_pca_clean[0], vectors_pca_clean[1])
#plt.scatter(x, y, marker='D', color='green')
plt.scatter(xc, yc, marker='*', color='red')

